# SNAP - NEXUSFORGE - Querying with SPARQL

SNAP and NEXUSFORGE also support querying data using SPARQL queries with `NexusHelper.get_entities_by_query`. This is considered a feature for advanced users and won't be thoroughly covered in these notebooks.

### Setup

The token can be found in the upper right corner on https://bbp.epfl.ch/nexus/web/ and pasted here

In [1]:
import getpass
from bluepysnap.nexus import NexusHelper

BUCKET = "nse/test"
TOKEN = getpass.getpass()
nexus = NexusHelper(BUCKET, TOKEN)

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

### Querying

#### Find resources by type

Let's start with a simple example of finding a resource by type by finding the `DetailedCircuit`s:

In [2]:
query = f"""
SELECT DISTINCT ?id WHERE {{
    ?id a DetailedCircuit.
}}
"""
records = nexus.get_entities_by_query(query)
print(f'Length: {len(records)}\nType: {type(records)}\n')

Length: 4
Type: <class 'list'>



Just like `NexusHelper.get_entities`, `NexusHelper.get_entities_by_query` returns a `list` (of `Entity` objects).

#### Filtering
With `NexusHelper.get_entities`, we searched for `DetailedCircut`s with the following filters:
```
    "name": "Thalamus microcircuit v3",
    "createdBy": "ivaska"
```

Let's see how this looks in SPARQL:

In [3]:
circuit_name = "Thalamus microcircuit v3"
creator = "https://bbp.epfl.ch/nexus/v1/realms/bbp/users/ivaska"
query = f"""
SELECT DISTINCT ?id WHERE {{
    ?id a DetailedCircuit;
    schema:name ?_circuit_name;
    nxv:createdBy ?_createdBy.
    
    FILTER(?_circuit_name = "{circuit_name}").
    FILTER(?_createdBy = <{creator}>).
}}
"""
records = nexus.get_entities_by_query(query)
print(f'Length: {len(records)}\n')

Length: 1



#### Find linked resources

Also not available in `NexusHelper.get_entities`, is the neat ability to find resources that are linked but 
* we are not sure how, or
* they may be linked in different ways and covering all cases would be cumbersome.

In SPARQL, this is achived with the `(<>|!<>)*` wild card. Basically it equals to: 
> _no matter how many steps (or what are the steps) but there's a link._

Let's see how we can use it in a SparQL query by querying for resources (by type) that are linked to resource for which we have an ID:

In [4]:
def linked_resources_query(resource_type, nexus_id):
    return f"""
    SELECT DISTINCT ?id WHERE {{
        ?id a {resource_type} ;
        nxv:deprecated ?_deprecated .
        {{ 
            <{nexus_id}> (<>|!<>)* ?id .
        }}
        UNION 
        {{
            ?id (<>|!<>)* <{nexus_id}> .
        }}
        FILTER (?_deprecated = 'false'^^xsd:boolean) .
    }}"""

This function returns a query that is, in fact, a union of two queries that cover the both directions:
* _resource with given id_ __&rarr;__ _wanted resource type_
* _wanted resource type_ __&rarr;__ _resource with given id_

The lines with `?_deprecated` are just to filter out the deprecated properties. This is done automatically when searching with `NexusHelper.get_entities`.

For an example, let's see how to find `DetailedCircuit`s that are linked to a certain `SimulationCampaign`:

In [5]:
resource_type = "DetailedCircuit"
simulation_campaign_id = 'https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/ivaska.SimulationCampaignNexusTask_entity_managemen_https___bbp_epfl__gpfs_bbp_cscs_c_4b174cc572'
query = linked_resources_query(resource_type, simulation_campaign_id)

records = nexus.get_entities_by_query(query)
print(f'Length: {len(records)}')

Length: 1


And here is the same going the other way around (find `SimulationCampaign`s linked to a circuit):

In [6]:
resource_type = "SimulationCampaign"
circuit_id = 'https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/O0-20180419'
records = nexus.get_entities_by_query(linked_resources_query(resource_type, circuit_id))
print(f'Length: {len(records)}')

Length: 5


#### Ordering (and limiting) results

One feature not available in `NexusHelper.get_entities` is ordering the results by given criterion.
This can be used together with `LIMIT` to, for example, only return the latest resource.

To give an example, let's find the newest `DetailedCircut`:

> **_NOTE:_** By default, `nexusforge` adds prefixes and certain other criteria to the query string and can not correctly handle `ORDER BY DESC` yet. We need to add `rewrite=False` to prevent the manipulation of the query string and add the prefixes manually.

In [7]:
query = """
PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX commonshapes: <https://neuroshapes.org/commons/>
PREFIX datashapes: <https://neuroshapes.org/dash/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX oa: <http://www.w3.org/ns/oa#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?id WHERE {
    ?id a nsg:DetailedCircuit;
    nxv:createdAt ?_createdAt
} ORDER BY DESC (?_createdAt)
"""
records = nexus.get_entities_by_query(query, rewrite=False, limit=1)
print(f'Length: {len(records)}\nType: {type(records)}\n')

Length: 1
Type: <class 'list'>

